# KNN_SVM_프로모션_효율예측분석

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [3]:
mem = pd.read_csv("./data/member.csv")
tran = pd.read_csv("./data/transaction.csv")

In [5]:
mem.head(2)

,id,recency,zip_code,is_referral,channel,conversion
0,906145,10,Surburban,0,Phone,0
1,184478,6,Rural,1,Web,0


In [6]:
tran.head(2)

,id,num_item,total_amount
0,906145,5,34000
1,906145,1,27000


* 쇼핑몰 고객 데이터, 프로모션 쿠폰을 발행하고 사용 여부 데이터 수집
* mem: 고객 id, 최근 방문일, 사는 지역, 추천여부, 주요접속채널, 쿠폰사용여부(target)
* tran: 고객id, 구매수량, 총 구매금액
* 전통적 마케팅 분석 방법 RFM 기법을 활용해 고객 데이터에서 파생변수 생성 후 분석
* R: Recency: 현재일 - 최근 구매일
* F: Frequency: 구매빈도
* M: Monetary : 구매 금액
* 종속변수: coversion => 고객이 프로모션에 반응 했는가? 1 = yes, 0 = no

# 거리기반의 알고리즘을 사용시 독립변수들 간의 단위를 꼭 맞춰 주어야 함
# 스케일링
* MinMaxScaler: 모든 숫자를 0-1 사이의 숫자로 변환 - 데이터 분포의 모양을 그대로 유지
* StandardScaler: 평균을 0, 표준편차를 1로 하는 정규분포 형태로 변환 - 데이터의 분포 모양이 정규 분포로 바뀜
* RobustScaler: 사분위수를 이용해서 데이터를 스케일링 - 데이터에 이상값이 있을 때 사용, 이상값에 영향을 최소화 

# 머신러닝 모델별 스케일러
* knn(최근접이웃): MinMaxScaler, 이상치가 있는 경우 RobustScaler
* SVM(서포트 벡터 머신): StandardScaler, 이상치가 있는 경우 RobustScaler
* Logistic Regression: Standard, 이상치가 있는 경우 RobustScaler
* Linear / Ridge / Lasso: StandardScaler, 이상치가 있는 경우 RobustScaler
* KMeans / DBSCAN: MinMax or Standard, 이상치가 있는 경우 RobustScaler
* DecisionTree, RandomForest, XGBoost: 스케일링 불필요, 이상치가 있는 경우에도 안 해도 됨
* Navie Bayes: 스케일링 불필요,